<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#塩基配列、ictvからのリファレンスあり" data-toc-modified-id="塩基配列、ictvからのリファレンスあり-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>塩基配列、ictvからのリファレンスあり</a></span><ul class="toc-item"><li><span><a href="#nexus-to-fasta" data-toc-modified-id="nexus-to-fasta-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>nexus to fasta</a></span></li><li><span><a href="#リファレンスツリーのための配列収集（Pfamからドメイン配列を取得）" data-toc-modified-id="リファレンスツリーのための配列収集（Pfamからドメイン配列を取得）-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>リファレンスツリーのための配列収集（Pfamからドメイン配列を取得）</a></span></li><li><span><a href="#ラベルデータの読み込み（作成：Final_dataset_phylogenetic_tree_label）" data-toc-modified-id="ラベルデータの読み込み（作成：Final_dataset_phylogenetic_tree_label）-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>ラベルデータの読み込み（作成：Final_dataset_phylogenetic_tree_label）</a></span></li><li><span><a href="#ウイルス属の名前をマニュアルで補正" data-toc-modified-id="ウイルス属の名前をマニュアルで補正-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>ウイルス属の名前をマニュアルで補正</a></span><ul class="toc-item"><li><span><a href="#known-viruses-vs-unknown-viruses" data-toc-modified-id="known-viruses-vs-unknown-viruses-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>known viruses vs unknown viruses</a></span></li></ul></li></ul></li><li><span><a href="#refseqをNCBI-Virus-DBから取得" data-toc-modified-id="refseqをNCBI-Virus-DBから取得-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>refseqをNCBI Virus DBから取得</a></span></li><li><span><a href="#アライメント作成" data-toc-modified-id="アライメント作成-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>アライメント作成</a></span><ul class="toc-item"><li><span><a href="#ICTVのリファレンスツリーを使用" data-toc-modified-id="ICTVのリファレンスツリーを使用-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>ICTVのリファレンスツリーを使用</a></span></li><li><span><a href="#Refseq,-GenBankのみから構築：今のところ失敗" data-toc-modified-id="Refseq,-GenBankのみから構築：今のところ失敗-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Refseq, GenBankのみから構築：今のところ失敗</a></span></li></ul></li></ul></div>

In [1]:
#!/usr/bin/env python
import sys, re
argvs = sys.argv
sys.setrecursionlimit(10000000)

import os
import collections
import numpy as np
import pandas as pd
import array
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 300)

import glob

## visualize
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

plt.rcParams['font.family'] = "Arial"

# Using reference alignment (nucleotide sequences) published by ictv

In [2]:
virus="Flaviviridae"

In [3]:
virus_l = [virus]


# read metadata 

In [10]:
metadata_f = "input_examples/final_dataset_phylogenetic_tree_label.txt"
df_label = pd.read_csv(metadata_f, sep="\t")
df_label.head(3)

,virus_ID,Family,database,blast,deepac,dnabert,kNN,vibe,zoonotic_rank,Genus,Species,refseq.id,dataset,label,host_label
0,AB031663,Flaviviridae,GenBank,1.0,0.902995,0.977958,1.0,0.922051,0.997712,Hepacivirus,Hepacivirus C,AB031663,past,1.0,1.0
1,AB049088,Flaviviridae,GenBank,1.0,0.997959,0.999127,1.0,0.986883,0.997712,Hepacivirus,Hepacivirus C,AB049088,past,1.0,1.0
2,AB114858,Flaviviridae,GenBank,0.0,0.044239,0.062404,0.1,0.094962,0.997712,Flavivirus,Yokose virus,AB114858,past,0.0,0.0


In [11]:
df_known = df_label[(df_label["dataset"] == "past") & 
                      (df_label["host_label"] == 1)]

genbank_known_l = df_known["refseq.id"].values.tolist()
output_dir = "/".join(map(str, ["output_examples", virus]))

print (len(genbank_known_l))

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

output_f = "/".join(map(str, [output_dir, virus + ".genbank_known_human.txt"]))

f = open(output_f, "w")
f.write("\n".join(map(str, genbank_known_l)))
f.close()

6010


In [12]:
df_known = df_label[(df_label["dataset"] == "past") 
                    & (df_label["host_label"] == 0)]

genbank_known_l = df_known["refseq.id"].values.tolist()
output_dir = "/".join(map(str, ["output_examples", virus]))

print (len(genbank_known_l))

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

output_f = "/".join(map(str, [output_dir, virus + ".genbank_known_animal.txt"]))

f = open(output_f, "w")
f.write("\n".join(map(str, genbank_known_l)))
f.close()

5162


In [13]:
df_unknown = df_label[df_label["dataset"] == "future"]

genbank_unknown_l = df_unknown["refseq.id"].values.tolist()
output_dir = "/".join(map(str, ["output_examples", virus]))

print (len(genbank_unknown_l))

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

output_f = "/".join(map(str, [output_dir, virus + ".genbank_unknown.txt"]))

f = open(output_f, "w")
f.write("\n".join(map(str, genbank_unknown_l)))
f.close()

1519


# mafft alignment

In [17]:
%%bash

virus="Flaviviridae"

cdhit_animal=0.95
cdhit_human=0.9
    
known_id_f=output_examples/${virus}/${virus}.genbank_known
unknown_id_f=output_examples/${virus}/${virus}.genbank_unknown
fasta_f=input_examples/${virus}/${virus}.curated.fasta
    

## extract past dataset sequences
for key in "animal" "human"
do
    cat ${fasta_f} | \
        seqkit grep -f ${known_id_f}_${key}.txt \
        > ${known_id_f}_${key}.fasta

    seq_n=$(seqkit stats ${known_id_f}_${key}.fasta | \
        awk 'NR==2 {print $4}' | sed 's/,//g')

    if [ ${seq_n} -gt 200 ] && [ ${key} == "animal" ];then
        cd-hit -i ${known_id_f}_${key}.fasta \
            -o ${known_id_f}_${key}.cdhit.fasta \
            -c 0.95 -n 5 -d 0 

    elif  [ ${seq_n} -gt 200 ] && [ ${key} == "human" ];then
        cd-hit -i ${known_id_f}_${key}.fasta \
            -o ${known_id_f}_${key}.cdhit.fasta \
            -c 0.9 -n 5 -d 0 

    else
            cp ${known_id_f}_${key}.fasta \
            ${known_id_f}_${key}.cdhit.fasta
    fi
done

## extract future dataset sequences
cat ${fasta_f} | \
    seqkit grep -f ${unknown_id_f}.txt \
    > ${unknown_id_f}.fasta

## combine the sequences
cat ${unknown_id_f}.fasta \
    ${known_id_f}_animal.cdhit.fasta \
    ${known_id_f}_human.cdhit.fasta \
    > output_examples/add.fasta

Program: CD-HIT, V4.8.1, Feb 22 2022, 08:58:29
Command: cd-hit -i
         output_examples/Flaviviridae/Flaviviridae.genbank_known_animal.fasta
         -o
         output_examples/Flaviviridae/Flaviviridae.genbank_known_animal.cdhit.fasta
         -c 0.95 -n 5 -d 0

Started: Thu May  2 17:54:08 2024
                            Output                              
----------------------------------------------------------------
total seq: 3008
longest and shortest : 23810 and 650
Total letters: 32333140
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 32M
Buffer          : 1 X 15M = 15M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 113M

Table limit with the given memory limit:
Max number of representatives: 279325
Max number of word counting entries: 85783443

comparing sequences from          0  to       3008
...
     3008  finished        440  clusters

Approximated maximum memory consumption: 117M
writing new database
w

[INFO] 5162 patterns loaded from file
[INFO] 6010 patterns loaded from file
[INFO] 1519 patterns loaded from file
/Users/junna/anaconda3/envs/env_210819/bin/mafft: Cannot open output_examples/ictv.fasta.

/Users/junna/anaconda3/envs/env_210819/bin/mafft: line 1255: /var/folders/ty/4bj553gd6mddwlrt75bs49yh0000gp/T/mafft.XXXXXXXXXX.NQp2bzwf/_addfile: No such file or directory
awk: can't open file /var/folders/ty/4bj553gd6mddwlrt75bs49yh0000gp/T/mafft.XXXXXXXXXX.NQp2bzwf/addsize
 source line number 1
/Users/junna/anaconda3/envs/env_210819/bin/mafft: line 1257: [: -eq: unary operator expected
/Users/junna/anaconda3/envs/env_210819/bin/mafft: line 1271: /var/folders/ty/4bj553gd6mddwlrt75bs49yh0000gp/T/mafft.XXXXXXXXXX.NQp2bzwf/infile: No such file or directory
awk: can't open file /var/folders/ty/4bj553gd6mddwlrt75bs49yh0000gp/T/mafft.XXXXXXXXXX.NQp2bzwf/size
 source line number 1
awk: can't open file /var/folders/ty/4bj553gd6mddwlrt75bs49yh0000gp/T/mafft.XXXXXXXXXX.NQp2bzwf/size
 source li

CalledProcessError: Command 'b'\nvirus="Flaviviridae"\n\ncdhit_animal=0.95\ncdhit_human=0.9\n    \nknown_id_f=output_examples/${virus}/${virus}.genbank_known\nunknown_id_f=output_examples/${virus}/${virus}.genbank_unknown\nfasta_f=input_examples/${virus}/${virus}.curated.fasta\n    \n\n## extract past dataset sequences\nfor key in "animal" "human"\ndo\n    cat ${fasta_f} | \\\n        seqkit grep -f ${known_id_f}_${key}.txt \\\n        > ${known_id_f}_${key}.fasta\n\n    seq_n=$(seqkit stats ${known_id_f}_${key}.fasta | \\\n        awk \'NR==2 {print $4}\' | sed \'s/,//g\')\n\n    if [ ${seq_n} -gt 200 ] && [ ${key} == "animal" ];then\n        cd-hit -i ${known_id_f}_${key}.fasta \\\n            -o ${known_id_f}_${key}.cdhit.fasta \\\n            -c 0.95 -n 5 -d 0 \n\n    elif  [ ${seq_n} -gt 200 ] && [ ${key} == "human" ];then\n        cd-hit -i ${known_id_f}_${key}.fasta \\\n            -o ${known_id_f}_${key}.cdhit.fasta \\\n            -c 0.9 -n 5 -d 0 \n\n    else\n            cp 

In [20]:
%%bash

virus="Flaviviridae"

cdhit_animal=0.95
cdhit_human=0.9
    
known_id_f=output_examples/${virus}/${virus}.genbank_known
unknown_id_f=output_examples/${virus}/${virus}.genbank_unknown
fasta_f=input_examples/${virus}/${virus}.curated.fasta
    

## mafft -add
mafft --auto --keeplength \
    --addfragments output_examples/add.fasta \
    --thread 4 \
    input_examples/${virus}/ictv.fasta \
    > output_examples/add.mafft.fasta

## alignment triming
trimal \
    -in output_examples/add.mafft.fasta \
    -out output_examples/add.mafft.trimal.fasta \
    -gt 0.8 -cons 80

## remove sequences where >80% of the total alignment length are gaps
python \
    script/seqkit_count_gap.py \
    -f output_examples/add.mafft.trimal.fasta -p 80

## extract sequence names in reference alignment
cat input_examples/${virus}/ictv.fasta | \
    grep ">" | sed 's/>//g' \
    > output_examples/${virus}/ictv.txt

## remove reference sequences
cat output_examples/add.mafft.trimal_rmgap.fasta | \
    seqkit grep -v -f output_examples/${virus}/ictv.txt \
    > output_examples/add.mafft.trimal.rmgap.out.fasta 


nadd =  1600
npair =  200000
nseq =  1725
nlen =  23810
use ktuples, size=6!
nadd = 1600
ppenalty_ex = -10
nthread = 4
blosum 62 / kimura 200
sueff_global = 0.100000
norg = 125
njobc = 126
generating a scoring matrix for nucleotide (dist=200) ... done


Making a distance matrix ..

There are 7507 ambiguous characters
  101 / 125 (thread    0)
done.

f
STEP 500 / 1600 (thread 3)                    
f
STEP 600 / 1600 (thread 0)                    
f
STEP 700 / 1600 (thread 2)                    
f
STEP 800 / 1600 (thread 1)                    
f
STEP 900 / 1600 (thread 2)                    
f
STEP 1000 / 1600 (thread 2)                    
f
STEP 1100 / 1600 (thread 2)                    
f
STEP 1200 / 1600 (thread 3)                    
f
STEP 1300 / 1600 (thread 1)                    
f
STEP 1400 / 1600 (thread 3)                    
f
STEP 1500 / 1600 (thread 0)                    
f

Combining ..
   done.                      

   done.                      

addsingle (nuc) Version

In [31]:
%%bash

virus="Flaviviridae"

cdhit_animal=0.95
cdhit_human=0.9
    
known_id_f=output_examples/${virus}/${virus}.genbank_known
unknown_id_f=output_examples/${virus}/${virus}.genbank_unknown
fasta_f=input_examples/${virus}/${virus}.curated.fasta


phylogenetic analysis
iqtree \
    -s output_examples/add.mafft.trimal.rmgap.out.fasta  \
    -m TESTNEW \
    -redo \
    -nt AUTO

IQ-TREE multicore version 2.1.4-beta COVID-edition for Mac OS X 64-bit built Jun 24 2021
Developed by Bui Quang Minh, James Barbetti, Nguyen Lam Tung,
Olga Chernomor, Heiko Schmidt, Dominik Schrempf, Michael Woodhams.

Host:    j-macstudio.nih.go.jp (SSE4.2, 128 GB RAM)
Command: iqtree -s output_examples/add.mafft.trimal.rmgap.out.fasta -m TESTNEW -redo -nt AUTO
Seed:    743578 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Thu May  2 20:10:12 2024
Kernel:  SSE2 - auto-detect threads (20 CPU cores detected)

Reading alignment file output_examples/add.mafft.trimal.rmgap.out.fasta ... Fasta format detected
Alignment most likely contains DNA/RNA sequences
Alignment has 1600 sequences with 870 columns, 870 distinct patterns
870 parsimony-informative, 0 singleton sites, 0 constant sites
           Gap/Ambiguity  Composition  p-value
   1  NC_074777    0.34%    failed      0.00%
   2  NC_074778    0.00%    passed     26.13%
   3  NC_076029    0.46%    passed      6.23%
  

bash: line 12: phylogenetic: command not found
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
